In [1]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import tensorflow as tf
#import matplotlib.pyplot as plt
from PIL import Image
from keras.layers import *
from keras.optimizers import *

2024-11-02 12:44:27.907851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 12:44:27.907895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 12:44:27.908620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 12:44:27.914068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 12:44:28.552022: W tensorflow/compiler/tf2

In [2]:
#!python3.9 -m pip install pillow
tf.config.list_physical_devices('GPU') 

2024-11-02 11:54:10.526501: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:10.550325: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:10.550634: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
errorCount = 0
DirectoryList = ['Anthurium (Anthurium andraeanum)',
 'Bird of Paradise (Strelitzia reginae)',
 'Boston Fern (Nephrolepis exaltata)',
 'Chinese Money Plant (Pilea peperomioides)',
 'Hyacinth (Hyacinthus orientalis)',
 'Monstera Deliciosa (Monstera deliciosa)',
 'Poinsettia (Euphorbia pulcherrima)',
 'Prayer Plant (Maranta leuconeura)',
 'Tradescantia']
for dirn in DirectoryList:
    for im in os.listdir(f"Data/{dirn}"):
        try:
            image = Image.open(f"Data/{dirn}/{im}")
            new_image = image.resize((128, 128)) 
            new_image.save(f"MoreScaled/{dirn}/{im}") 
        except Exception as e:
            print(e)
            print("Error with image")
            errorCount += 1
    print(f"Total of {errorCount} errors")

cannot write mode RGBA as JPEG
Error with image
Total of 1 errors
Total of 1 errors
cannot write mode RGBA as JPEG
Error with image
Total of 2 errors
cannot write mode RGBA as JPEG
Error with image
cannot write mode RGBA as JPEG
Error with image
Total of 4 errors
Total of 4 errors
cannot write mode RGBA as JPEG
Error with image
Total of 5 errors
cannot write mode RGBA as JPEG
Error with image
Total of 6 errors
Total of 6 errors
Total of 6 errors


In [5]:
for dirnam in os.listdir("Data/"):
    os.mkdir(f"MoreScaled/{dirnam}")

In [2]:
image_size = (256, 256)
batch_size = 4

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "Scaled",
    validation_split=0.2,
    label_mode='categorical',
    subset="both",
    seed=99,
    image_size=image_size,
    batch_size=batch_size,
)

Found 3187 files belonging to 9 classes.
Using 2550 files for training.
Using 637 files for validation.


2024-11-02 12:44:34.867008: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 12:44:34.886110: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 12:44:34.886411: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 12:44:34.888748: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 12:44:34.889150: I external/local_xla/xla/stream_executor

In [3]:
model = keras.Sequential()
model.add(Input(shape=(256, 256, 3)))
model.add(Rescaling(1.0/255))
model.add(RandomBrightness(.4, value_range=[0.0, 1.0]))
model.add(RandomFlip(mode="vertical"))
model.add(RandomContrast(.3, seed=None))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.75))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation(tf.nn.softmax))

In [4]:
#sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(learning_rate=0.0007)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy', 'CategoricalCrossentropy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 random_brightness (RandomB  (None, 256, 256, 3)       0         
 rightness)                                                      
                                                                 
 random_flip (RandomFlip)    (None, 256, 256, 3)       0         
                                                                 
 random_contrast (RandomCon  (None, 256, 256, 3)       0         
 trast)                                                          
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 activation (Activation)     (None, 256, 256, 32)      0

In [5]:
model.fit(train_ds, epochs=20, validation_data=val_ds)

Epoch 1/20


2024-11-02 12:45:05.108835: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-11-02 12:45:05.775002: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-11-02 12:45:05.884139: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2024-11-02 12:45:06.707516: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f790e2865b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-02 12:45:06.707547: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2024-11-02 12:45:06.711994: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash rep

420/638 [==================>...........] - ETA: 5s - loss: 2.7436 - accuracy: 0.2542 - categorical_crossentropy: 2.7436

2024-11-02 12:45:17.002518: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


524/638 [=======================>......] - ETA: 2s - loss: 2.5697 - accuracy: 0.2576 - categorical_crossentropy: 2.5697

...

638/638 [==============================] - 15s 24ms/step - loss: 0.8163 - accuracy: 0.7157 - categorical_crossentropy: 0.8163 - val_loss: 1.4305 - val_accuracy: 0.5338 - val_categorical_crossentropy: 1.4305
Epoch 20/20
421/638 [==================>...........] - ETA: 4s - loss: 0.7938 - accuracy: 0.7090 - categorical_crossentropy: 0.7938

2024-11-02 12:50:19.192216: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


526/638 [=======================>......] - ETA: 2s - loss: 0.8145 - accuracy: 0.7025 - categorical_crossentropy: 0.8145

2024-11-02 12:50:21.531734: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


638/638 [==============================] - 15s 24ms/step - loss: 0.8109 - accuracy: 0.7043 - categorical_crossentropy: 0.8109 - val_loss: 1.4265 - val_accuracy: 0.5228 - val_categorical_crossentropy: 1.4265


In [6]:
model.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
421/638 [==================>...........] - ETA: 4s - loss: 0.7783 - accuracy: 0.7262 - categorical_crossentropy: 0.7783

2024-11-02 12:50:34.757616: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


526/638 [=======================>......] - ETA: 2s - loss: 0.7720 - accuracy: 0.7281 - categorical_crossentropy: 0.7720

2024-11-02 12:50:37.096637: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


638/638 [==============================] - 16s 24ms/step - loss: 0.7746 - accuracy: 0.7263 - categorical_crossentropy: 0.7746 - val_loss: 1.4206 - val_accuracy: 0.5510 - val_categorical_crossentropy: 1.4206
Epoch 2/30
421/638 [==================>...........] - ETA: 4s - loss: 0.7305 - accuracy: 0.7476 - categorical_crossentropy: 0.7305

...

638/638 [==============================] - 16s 25ms/step - loss: 0.4912 - accuracy: 0.8435 - categorical_crossentropy: 0.4912 - val_loss: 1.8490 - val_accuracy: 0.5338 - val_categorical_crossentropy: 1.8490
Epoch 30/30
420/638 [==================>...........] - ETA: 5s - loss: 0.4514 - accuracy: 0.8488 - categorical_crossentropy: 0.4514

2024-11-02 12:58:12.180392: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


524/638 [=======================>......] - ETA: 2s - loss: 0.4442 - accuracy: 0.8483 - categorical_crossentropy: 0.4442

2024-11-02 12:58:14.588549: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


638/638 [==============================] - 16s 25ms/step - loss: 0.4479 - accuracy: 0.8478 - categorical_crossentropy: 0.4479 - val_loss: 2.0132 - val_accuracy: 0.5275 - val_categorical_crossentropy: 2.0132


In [7]:
model.save("best.weights")

INFO:tensorflow:Assets written to: best.weights/assets


INFO:tensorflow:Assets written to: best.weights/assets
